In [16]:
!git pull

remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/basilwong/awstest1
   5147051..d1d81de  master     -> origin/master
Updating 5147051..d1d81de
Fast-forward
 ffmpeg-release-amd64-static.tar.xz | Bin 0 -> 37907552 bytes
 final-output/final_audio.mp3       | Bin 3955400 -> 3955400 bytes
 final-output/vocals.mp3            | Bin 1390882 -> 900198 bytes
 3 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 ffmpeg-release-amd64-static.tar.xz


### Add Dependencies

In [ ]:
import sagemaker as sage
from sagemaker import get_execution_role

import zipfile
import os

from sagemaker import ModelPackage

# Installing src dependency.
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('src')

!pip install pydub
import audio_util
import processing_util

#Installing ffmpeg dependency. (https://linuxize.com/post/how-to-install-ffmpeg-on-centos-7/)
# !sudo yum install epel-release
# !sudo rpm -v --import http://li.nux.ro/download/nux/RPM-GPG-KEY-nux.ro
# !sudo rpm -Uvh http://li.nux.ro/download/nux/dextop/el7/x86_64/nux-dextop-release-0-5.el7.nux.noarch.rpm
!sudo yum localinstall --nogpgcheck https://download1.rpmfusion.org/free/el/rpmfusion-free-release-6.noarch.rpm https://download1.rpmfusion.org/nonfree/el/rpmfusion-nonfree-release-6.noarch.rpm
!sudo yum install ffmpeg ffmpeg-devel
ffmpeg -version



In [4]:
# Execution role
role = get_execution_role()
# S3 prefixes
common_prefix = "source_separation"
batch_inference_input_prefix = common_prefix + "/batch-inference-input-data"
# Sagemaker Session
sagemaker_session = sage.Session()
# Arn for Source Separator Model Package
modelpackage_arn = 'arn:aws:sagemaker:us-east-2:057799348421:model-package/source-separation-v11570291536-75ed8128ecee95e142ec4404d884ecad'



### Creating the Model

In [3]:
from sagemaker import ModelPackage

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='application/x-recordio-protobuf')

model = ModelPackage(role=role,
                     model_package_arn=modelpackage_arn,
                     sagemaker_session=sagemaker_session,
                     predictor_cls=predict_wrapper)

### Running the Batch Job

Note that if the initial audio file is longer than around 30 seconds, it is too large for the model. The split_mp3() method in  src.audio_util works around this by splitting an mp3 file into 30 second segments. 

In [44]:
# !apt
audio_util.split_mp3("songs/drake-toosie_slide.mp3", "../source-separation-input/")

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pydub/utils.py:193: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [Errno 2] No such file or directory: 'ffprobe': 'ffprobe'

In [4]:
batch_input_folder = "source-separation-input"


transform_input = sagemaker_session.upload_data(batch_input_folder, key_prefix=batch_inference_input_prefix)
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-075178354542/source_separation/batch-inference-input-data


In [5]:
import json 
import uuid

bucket = sagemaker_session.default_bucket()

transformer = model.transformer(1, 'ml.m4.xlarge', strategy='SingleRecord', output_path='s3://'+bucket+'/'+common_prefix+'/batch-transform-output')
transformer.transform(transform_input, content_type='application/x-recordio-protobuf')
transformer.wait()

print("Batch Transform output saved to " + transformer.output_path)

....................Starting the inference server with 4 workers.
[2020-04-13 04:48:15 +0000] [10] [INFO] Starting gunicorn 19.9.0
[2020-04-13 04:48:15 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
[2020-04-13 04:48:15 +0000] [10] [INFO] Using worker: gevent
[2020-04-13 04:48:15 +0000] [14] [INFO] Booting worker with pid: 14
[2020-04-13 04:48:15 +0000] [15] [INFO] Booting worker with pid: 15
[2020-04-13 04:48:15 +0000] [16] [INFO] Booting worker with pid: 16
[2020-04-13 04:48:15 +0000] [17] [INFO] Booting worker with pid: 17
Testing...
Testing...
2020-04-13 04:48:39.079016: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
169.254.255.130 - - [13/Apr/2020:04:48:39 +0000] "GET /ping HTTP/1.1" 200 1 "-" "Go-http-client/1.1"
169.254.255.130 - - [13/Apr/2020:04:48:39 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753319

['audio_file_1586753595.1332912.mp3_vocals.wav', 'audio_file_1586753595.1332912.mp3_accompaniment.wav']
['audio_file_1586753595.1332912.mp3_vocals.wav', 'audio_file_1586753595.1332912.mp3_accompaniment.wav']
169.254.255.130 - - [13/Apr/2020:04:54:01 +0000] "POST /invocations HTTP/1.1" 200 19459102 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753641.1667993.mp3
Producing source estimates for input mixture file /tmp/audio_file_1586753641.1667993.mp3
Testing...
169.254.255.130 - - [13/Apr/2020:04:54:01 +0000] "POST /invocations HTTP/1.1" 200 19459102 "-" "Go-http-client/1.1"
Input path : /tmp/audio_file_1586753641.1667993.mp3
Producing source estimates for input mixture file /tmp/audio_file_1586753641.1667993.mp3
Testing...
Num of variables64
Pre-trained model restored for song prediction
Num of variables64
Pre-trained model restored for song prediction
169.254.255.130 - - [13/Apr/2020:04:54:47 +0000] "POST /invocations HTTP/1.1" 200 19035684 "-" "Go-http-client/1.1"
Input pa

### Processing the Batch Output

In [142]:
import boto3
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "source_separation/batch-transform-output/"
i = 0
audio_util.clear_folder('source-separation-output/batch-transform-output')
for object_summary in my_bucket.objects.filter(Prefix=prefix):
    i = i + 1
    file_name = object_summary.key.split('/')[-1]
    print(file_name)
    my_bucket.download_file(prefix+ file_name, 'source-separation-output/batch-transform-output/output-{}.zip'.format(i))

input1.mp3.out
input2.mp3.out
input3.mp3.out
input4.mp3.out
input5.mp3.out
input6.mp3.out
input7.mp3.out
input8.mp3.out
input9.mp3.out


In [148]:
# Extracting files from zip files. 
audio_util.clear_folder('source-separation-output/extracted')
for file in os.listdir('source-separation-output/batch-transform-output'):
    print(file)
    with zipfile.ZipFile('source-separation-output/batch-transform-output/'+file, 'r') as zip_ref:
        zip_ref.extractall('source-separation-output/extracted/'+file.split('.')[0]+'/')

output-3.zip
output-6.zip
output-8.zip
output-9.zip
output-1.zip
output-4.zip
output-7.zip
output-2.zip
output-5.zip


In [150]:
import os
# Separating the vocal files and the background sound files.
audio_util.clear_folder('source-separation-output/vocals')
audio_util.clear_folder('source-separation-output/background')
for i, folder in enumerate(sorted(os.listdir('source-separation-output/extracted/'))):
    for file in os.listdir('source-separation-output/extracted/' + folder + '/output'):
        new_file_name = str(i).zfill(5) + ".wav"
        if "vocals" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/vocals/vocals' + new_file_name)
        elif "accompaniment" in file:
            os.rename('source-separation-output/extracted/' + folder + '/output/' + file, 'source-separation-output/background/background' + new_file_name)

### Transcribe the Vocal Files

In [151]:
# Upload the Vocal files onto s3
local_vocals_folder = "source-separation-output/vocals/"
transcribe_input_prefix = "transcribe-input"

transcribe_input = sagemaker_session.upload_data(local_vocals_folder, key_prefix=transcribe_input_prefix)
print("Transcribe input uploaded to " + transcribe_input)

Transcribe input uploaded to s3://sagemaker-us-east-2-075178354542/transcribe-input


In [ ]:
# Start a transcription job for each file. Add the transcription to finsihed jobs once finished. 

import boto3
from datetime import datetime

transcribe = boto3.client('transcribe')
output_bucket_name = "transcribe-output"
audio_util.clear_folder('transcribe-output')
uri_prefix = "https://sagemaker-us-east-2-075178354542.s3.us-east-2.amazonaws.com/transcribe-input/"
finished_jobs = list()

for file in sorted(os.listdir(local_vocals_folder)):

    print("Transcribing: " + file)
    job_uri = uri_prefix + file
    transcribe.start_transcription_job(
        TranscriptionJobName=file,
        Media={'MediaFileUri': job_uri},
        MediaFormat='wav',
        LanguageCode='en-US'
    )
    while True:
        status = transcribe.get_transcription_job(TranscriptionJobName=file)
        if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
    
    api_data = requests.get(url=status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = api_data.json()
    finished_jobs.append(data)
    dump_file_name = 'transcribe-output/transcription' + file.split(".")[0] + '.json'
    # Writing to json files for analysis purposes.
    with open(dump_file_name, 'w') as f:
        json.dump(data, f, indent=4)
    transcribe.delete_transcription_job(TranscriptionJobName=file)
    
finished_jobs.sort(key=lambda x : x['jobName'])

Transcribing: vocals00000.wav
Transcribing: vocals00001.wav


In [158]:
transcribe.delete_transcription_job(TranscriptionJobName="vocals00000.wav")

{'ResponseMetadata': {'RequestId': '20153835-a1cb-4d8b-b31b-fc5194cf9a11',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 13 Apr 2020 23:21:58 GMT',
   'x-amzn-requestid': '20153835-a1cb-4d8b-b31b-fc5194cf9a11',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Giving Transcriptions to Amazon Polly

In [22]:
def query_polly(polly_client, word, length, prefix, output_folder):
    
    ssml = """<speak><prosody amazon:max-duration="{max_len}ms">{word}</prosody></speak>""".format(max_len=str(length), word=word)          
    response = polly_client.start_speech_synthesis_task(VoiceId='Joey',
                OutputS3BucketName='sagemaker-us-east-2-075178354542',
                OutputS3KeyPrefix='polly-output/' + prefix,
                OutputFormat='mp3', 
                TextType = 'ssml',
                Text = ssml)


In [25]:
import json
# Patching the batches back together, generate transcription list from all the batches. 
transcribe_output_folder = "transcribe-output/"
offset = 0
transcription_list = list()
for file in sorted(os.listdir(transcribe_output_folder)):
    transcription_batch = json.load(open(transcribe_output_folder + file, "r", encoding="utf-8"))
    for map_item in transcription_batch["results"]["items"]:
        transcribe_object = processing_util.TranscriptionItem(map_item, offset)
        # Skip punctuation
        if transcribe_object.is_word():
            transcription_list.append(transcribe_object)
    offset += 30000

In [26]:
import boto3
polly_client = boto3.client('polly')

transcribe_output_folder = "transcribe-output/"
polly_output_folder = "polly-output/"
audio_util.clear_folder(polly_output_folder)
silence_dict = {"length" : 0}
index = 0
expected_start_time = 0

for i, transcribe_object in enumerate(transcription_list):
    
    # Create pause if there is a larger than 1 second gap between words.
    if expected_start_time + 1000 <= transcribe_object.start_time:
        silence_dict["length"] = transcribe_object.start_time - expected_start_time
        with open(polly_output_folder + str(index).zfill(5) + ".json", 'w') as outfile:
            json.dump(silence_dict, outfile)
        expected_start_time = transcribe_object.start_time
        index += 1
    
    # Extend the word if there is less than a 1 second gap between words.
    if i < len(transcription_list) - 1 and transcribe_object.end_time + 1000 > transcription_list[i + 1].start_time:
        transcribe_object.end_time = transcription_list[i + 1].start_time
        
    response = query_polly(polly_client, transcribe_object.content, transcribe_object.duration(), str(index).zfill(5), polly_output_folder)
    expected_start_time = transcribe_object.end_time
    print("Polly Queried for: " + transcribe_object.content)    
    index += 1

Polly Queried for: Oh
Polly Queried for: black
Polly Queried for: leather
Polly Queried for: blood
Polly Queried for: No
Polly Queried for: seafood
Polly Queried for: buckles
Polly Queried for: on
Polly Queried for: a
Polly Queried for: jacket
Polly Queried for: It's
Polly Queried for: a
Polly Queried for: leak
Polly Queried for: Shit
Polly Queried for: Nike
Polly Queried for: cross
Polly Queried for: body
Polly Queried for: got
Polly Queried for: a
Polly Queried for: piece
Polly Queried for: and
Polly Queried for: got
Polly Queried for: a
Polly Queried for: dance
Polly Queried for: But
Polly Queried for: it's
Polly Queried for: really
Polly Queried for: also
Polly Queried for: street
Polly Queried for: shoot
Polly Queried for: I'm
Polly Queried for: gonna
Polly Queried for: show
Polly Queried for: you
Polly Queried for: how
Polly Queried for: to
Polly Queried for: get
Polly Queried for: it
Polly Queried for: go
Polly Queried for: right
Polly Queried for: for
Polly Queried for: that
Po

Polly Queried for: move
Polly Queried for: on
Polly Queried for: Shaky
Polly Queried for: with
Polly Queried for: do
Polly Queried for: this
Polly Queried for: shit
Polly Queried for: ourselves
Polly Queried for: But
Polly Queried for: if
Polly Queried for: I'm
Polly Queried for: over
Polly Queried for: shake
Polly Queried for: each
Polly Queried for: other
Polly Queried for: due
Polly Queried for: to
Polly Queried for: shit
Polly Queried for: himself
Polly Queried for: Your
Polly Queried for: solo
Polly Queried for: figures
Polly Queried for: on
Polly Queried for: a
Polly Queried for: goal
Polly Queried for: O'Farrill
Polly Queried for: hurt
Polly Queried for: a
Polly Queried for: lot
Polly Queried for: of
Polly Queried for: budget
Polly Queried for: but
Polly Queried for: we
Polly Queried for: don't
Polly Queried for: know
Polly Queried for: for
Polly Queried for: next
Polly Queried for: time
Polly Queried for: Currency
Polly Queried for: the
Polly Queried for: true
Polly Queried for

### Processing the Output from Amazon Polly

In [48]:
import boto3
# Downloading files from s3.
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(sagemaker_session.default_bucket())
prefix = "polly-output/"

for object_summary in my_bucket.objects.filter(Prefix=prefix):
    file_name = object_summary.key.split('/')[-1]
    my_bucket.download_file(prefix+ file_name, prefix + file_name)
    
print("Files moved from s3 to repo.")

Files moved from s3 to repo.


In [193]:
print(index)

1036


Mixing Audio:

https://stackoverflow.com/questions/7629873/how-do-i-mix-audio-files-using-python

Pitch Modulation:

https://stackoverflow.com/questions/38923438/does-pydub-support-pitch-modulation

